## Basic Chat Model

In [1]:
#invoke through chat modesl

from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

chat_model = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai")

sys_msg = SystemMessage("System. You are professional translator, translating the text from user "
                        "from english  to chinese ")
human_msg = HumanMessage("Good Morning!")

chat_model.invoke([sys_msg, human_msg])

AIMessage(content='早上好！ (Zǎoshang hǎo!)\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 21, 'candidates_token_count': 13, 'total_token_count': 34, 'prompt_tokens_details': [{'modality': 1, 'token_count': 21}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 13}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.24137005439171424}, id='run-18dcbd6b-cb40-411f-8aa9-7743cb923cbe-0', usage_metadata={'input_tokens': 21, 'output_tokens': 13, 'total_tokens': 34})

In [2]:
# through llm directly.
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(
    temprature=0,
    model="gemini-2.0-flash-001"
)
llm.invoke([sys_msg, human_msg])


AIMessage(content='早上好！ (Zǎoshang hǎo!)\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 21, 'candidates_token_count': 13, 'total_token_count': 34, 'prompt_tokens_details': [{'modality': 1, 'token_count': 21}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 13}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.02032931034381573}, id='run-b2909991-fbc5-475c-807b-685bf0bd6463-0', usage_metadata={'input_tokens': 21, 'output_tokens': 13, 'total_tokens': 34})

In [19]:
# through chain with template

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_vertexai import ChatVertexAI
prompt = ChatPromptTemplate([
    ('system', 'You are professional translator, translating the user text from {from_lang} to {to_lang}'),
    ('user', '{msg}')
])

translator = ChatVertexAI(model="gemini-2.0-flash-001")

chain = prompt|translator
ai_msg=chain.invoke({
    'from_lang': 'English',
    'to_lang': 'Chinese',
    'msg': 'Good Morning'
})

print(ai_msg.content)

# stream
for ai_msg_chunk in  chain.stream({'from_lang': 'English',
    'to_lang': 'Chinese',
    'msg': 'Wheather is good today. I should go for a walking.'}):
    print(ai_msg_chunk.content, "---")

    

早上好 (Zǎo shang hǎo)

今天 ---
天气不错，我应该去散个步。
 ---


In [8]:
## Document, Splitter, vector store, Retriever.

# like message, prompt, document is a basic concept in langchain_core
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


pdf_loader = PyPDFLoader("~/Downloads/Accident insurance benefit- Google Copy.pdf")
docs = pdf_loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
splitts = splitter.split_documents(docs)



In [5]:
from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model="text-embedding-004")

embeds = embeddings.embed_documents(["Good morning", "Beijing is the capital city of China", 
                                    "Washington DC is the capital city of USA"])



In [12]:
from langchain_chroma  import Chroma
vector_store=Chroma(embedding_function=embeddings)
ids = vector_store.add_documents(documents=splitts)

In [13]:
results = vector_store.similarity_search("how much does it cost monthly?", k=2)

In [19]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":1})
results=retriever.batch(["How much is the montly premium?", "Is fracture covered?"])

In [34]:
llm = ChatVertexAI(model="gemini-2.0-flash-001")

from langchain_core.prompts import ChatPromptTemplate
from IPython.display import display, Markdown, Latex

prompt = ChatPromptTemplate([
    ('system', "you are a helpful insurance agent answering customer question related to insuarnce. Here is the insurance policy:\n {policy}"),
    ('user', 'How much should I pay for this policy monthly?')
])

prompt_text =  prompt.invoke({'policy': results[0][0].page_content})
ai_msg=llm.invoke(prompt_text)
display(Markdown(ai_msg.content)

AIMessage(content="Okay, to figure out your monthly cost, I need to know who you want to cover under the policy. Here's a breakdown of the monthly costs based on coverage:\n\n*   **Googler only:** $8.81\n*   **Googler and spouse/domestic partner:** $12.56\n*   **Googler and child(ren):** $17.71\n*   **Googler, spouse/domestic partner and child(ren):** $23.87\n\nAlso, keep in mind that your eligible dependent parents are automatically covered at 50 percent of your elected coverage.\n\nTell me who you want to cover, and I can tell you the exact monthly cost.", additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 310, 'candidates_token_count': 154, 'total_token_count': 464, 'prompt_tokens_details': [{'modality': 1, 'token_count': 310}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 154}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.0473299924

In [30]:
prompt.invoke({"policy": "good policy"})

ChatPromptValue(messages=[SystemMessage(content='system, you are a helpful insurance agent answering customer question related to insuarnce. Here is the insurance policy:\n{policy}', additional_kwargs={}, response_metadata={}), HumanMessage(content='How much should I pay for this policy monthly?', additional_kwargs={}, response_metadata={})])

In [29]:
results[0][0].page_content

'4Do you have aging parents?\nYour eligible dependent parents are automatically covered at 50 percent  \nof your elected coverage.\nMonthly cost of coverage\nGoogler only $8.81\nGoogler and spouse/domestic partner $12.56\nGoogler and child(ren) $17.71\nGoogler, spouse/domestic partner and child(ren) $23.87\nAll rates are subject to change.\nReady to enroll? \nIt’s quick and easy to enroll without answering health questions or  \na doctor’s exam. \nYou can enroll:\nWithin 31 days of initial eligibility period  \nDuring your open enrollment window\nWithin 31 days of a qualified family status change\nWe’re here to help\nAccident insurance questions? \nCall 833-745-5516  to chat with a Securian Financial customer service \nrepresentative, Monday-Friday 5 a.m.-5 p.m. Pacific Time.  \nEmail Securian Financial at googlesupport@securian.com  or sign up  \nto talk to a Securian concierge live at go/illnessandaccidentinsuranceoh .\nEnrollment questions? \nVisit go/benefits-support .\nLearn more'